In [ ]:
from tensorflow.keras.utils import save_img
import tensorflow_probability as tfp
from matplotlib import pyplot as plt
from os import mkdir, path
from keras import backend as K
import tensorflow as tf
import numpy as np
import pathlib
import glob
import cv2


DATASET_PATH = 'task_2/dataset/processed_hk_norm_unenhanced_aug_iris_dataset_64x240_png/'
MODELS_PATH = 'task_2/cnn-based_dfe/models/'

models_list = ['best_unet_bs4_dice_loss']

IMG_WIDTH = 240
IMG_HEIGHT = 64
LENGTH_IMAGE_NAME = 12 

# ---- ridurre la dimensione del testsize se ho usato anche il validation ---- #
testset_files = glob.glob(DATASET_PATH + 'test/*.png')
testset_files.sort()

new_test_files = []
len_file = len(testset_files[1])

i=0
for file in testset_files:
    parts = testset_files[i].split('/')
    file_name = parts[-1]
    sub = file_name[0:3]
    
    if int(sub) > 180 :
        new_test_files.append(file)
    i = i+1
    
testset_files = new_test_files
testset_size = len(testset_files)

PATHLIB_DATASET_PATH  = pathlib.Path(DATASET_PATH)

In [ ]:
def load(image_file):
    # Read and decode an image file to a uint8 tensor
    image = tf.io.read_file(image_file)

    image = tf.io.decode_png(image)

    # Split each image tensor into two tensors:
    # - one with a real building facade image
    # - one with an architecture label image 
    w = tf.shape(image)[1]
    w = w // 2

    input_image = image[:, :w, :]
    real_image = image[:, w:, :]

    # Convert both images to float32 tensors
    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)
        
    return input_image, real_image

In [ ]:
def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    real_image = tf.image.resize(real_image, [height, width],
                        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    return input_image, real_image

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1
         
    return input_image, real_image

def normalize2(input_image, real_image) :
    
    mean = tf.math.reduce_mean(input_image)
    std = tf.math.reduce_std(input_image)
    input_image = tf.clip_by_value((input_image - mean) / std, clip_value_min=-1.0, clip_value_max=1.0)

    mean = tf.math.reduce_mean(real_image)
    std = tf.math.reduce_std(real_image)
    real_image = tf.clip_by_value((real_image - mean) / std, clip_value_min=-1.0, clip_value_max=1.0)
    
    return input_image, real_image

In [ ]:
def generate_images(model, test_input, tar):
    prediction = model(test_input, training=False)
    rbg_prediction = cv2.cvtColor(prediction[0].numpy(), cv2.COLOR_BGR2GRAY)
    rbg_tar = cv2.cvtColor(tar[0].numpy(), cv2.COLOR_BGR2GRAY)
    
    plt.figure(figsize=(15, 15))

    display_list = [test_input[0], rbg_tar, rbg_prediction]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # Getting the pixel values in the [0, 1] range to plot.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    
    plt.show()

In [ ]:
# Daugman feature extraction algorithm

def tf_ProcessSingleChannel(channel):
    h = tf.histogram_fixed_width(channel, value_range=(0, 255), nbins=256)

    h = tf.cast(h, tf.float32)
    pixel_values = tf.range(256, dtype=tf.float32)
    
    weighted_sum = tf.reduce_sum(pixel_values * h)
    total_pixels = tf.reduce_sum(h)
    mean_val = weighted_sum / total_pixels

    # Compute variance and standard deviation
    variance = tf.reduce_sum(((pixel_values - mean_val) ** 2) * h) / total_pixels
    std_dev = tf.sqrt(variance)

    # Compute Gaussian values
    gaussian_vals = (1 / (std_dev * tf.sqrt(2 * np.pi))) * tf.exp(-0.5 * ((pixel_values - mean_val) / std_dev) ** 2)

    # Set threshold
    threshold = tf.reduce_max(gaussian_vals) * 0.1  # For example, 10% of the maximum

    # Find values to eliminate
    to_eliminate = gaussian_vals < threshold

    ProcessedChannel = tf.identity(channel)  # Create a copy

    # Replace values below the threshold
    for i in range(len(to_eliminate)):
        if to_eliminate[i]:
            ProcessedChannel = tf.where(channel == i, mean_val + std_dev, ProcessedChannel)

    return ProcessedChannel

def tf_GaussHistCut(image):
    channels = 1
    if len(image.shape) > 2:
        _, _, channels = image.shape

    if channels == 3:  # RGB image
        CorrectedImage = tf.zeros_like(image, dtype=tf.uint8)

        for ch in range(channels):
            CorrectedImage[:, :, ch] = tf_ProcessSingleChannel(image[:, :, ch])
    
    else:  # Grayscale image
        CorrectedImage = tf_ProcessSingleChannel(image)

    return CorrectedImage

def tf_rescale(data):
    data_min = tf.reduce_min(data)
    data_max = tf.reduce_max(data)
    return (data - data_min) / (data_max - data_min)

def tf_mad_normalize(channel):
    mad = tfp.stats.percentile(tf.abs(channel - tfp.stats.percentile(channel, 50)), 50)
    is_zero_mad = tf.equal(mad, 0)
    channel = tf.where(is_zero_mad, tf.zeros_like(channel), (channel - tfp.stats.percentile(channel, 50)) / mad)
    return tf_rescale(channel)

def tf_daugman_normalization(image):

    AR, AG, AB = tf.split(image, num_or_size_splits=3, axis=-1)

    # Apply GaussHistCut
    AR = tf_GaussHistCut(AR)
    #AG = tf_GaussHistCut(AG)
    #AB = tf_GaussHistCut(AB)

    AR = tf_mad_normalize(AR)
    #AG = tf_mad_normalize(AG)
    #AB = tf_mad_normalize(AB)

    # Replace NaN and Inf values with 0
    AR = tf.where(tf.math.is_nan(AR) | tf.math.is_inf(AR), tf.zeros_like(AR), AR)
    #AG = tf.where(tf.math.is_nan(AG) | tf.math.is_inf(AG), tf.zeros_like(AG), AG)
    #AB = tf.where(tf.math.is_nan(AB) | tf.math.is_inf(AB), tf.zeros_like(AB), AB)

    # Create the normalized image
    #norm_image = tf.concat([AR, AG, AB], axis=-1)

    return AR #return norm_image
    
def tf_gaborconvolve(im, nscale, minWaveLength, mult, sigmaOnf):
    rows = IMG_HEIGHT #im.shape[0]
    cols = IMG_WIDTH #im.shape[1]
    
    filtersum = tf.zeros(cols, dtype=tf.float32)
    EO = [None] * nscale
    
    ndata = cols

    logGabor = tf.zeros(ndata, dtype=tf.float32)
    result = tf.zeros([rows, ndata], dtype=tf.complex128)
    
    radius = tf.range(0, ndata // 2 + 1, dtype=tf.float64) / (ndata // 2) / 2  # Frequency values 0 - 0.5
    zerovalue = tf.cast(tf.constant([1.0]), dtype=tf.float64)
    radius = tf.tensor_scatter_nd_update(radius, tf.constant([[0]]), zerovalue)
    
    wavelength = minWaveLength  # Initialize filter wavelength
    
    for s in range(nscale):
        # Construct the filter - first calculate the radial filter component
        fo = 1.0 / wavelength  # Centre frequency of filter
        # corresponding to fo
        
        sum = tf.exp( tf.cast( - tf.pow((tf.math.log(radius/fo)), 2), dtype=tf.float32) / (2 * tf.pow(tf.math.log(sigmaOnf), 2)))


        indexes = tf.expand_dims(tf.range(0, sum.shape[0]), axis=1)

        logGabor = tf.tensor_scatter_nd_update(logGabor, indexes, sum)
        logGabor = tf.tensor_scatter_nd_update(logGabor, tf.constant([[0]]), tf.constant([0.0]))
        
        filter = logGabor
        filtersum = filtersum + filter
        
        for r in range(rows):
            signal = im[r, 0:ndata]
            imagefft = tf.signal.fft(tf.cast(signal, dtype=tf.complex128))
            filter = tf.cast(filter, dtype=tf.complex128)
            result = tf.tensor_scatter_nd_add(result, [tf.constant([r])], [tf.signal.ifft(imagefft * filter)])
        
        EO[s] = result
        wavelength *= mult  # Finally calculate the wavelength of the next filter
    
    filtersum = tf.signal.fftshift(filtersum)
    
    return EO, filtersum

def tf_encode(polar_array, nscales, minWaveLength, mult, sigmaOnf):
    # Convoluzione della regione normalizzata con filtri di Gabor
    E0, _ = tf_gaborconvolve(polar_array, nscales, minWaveLength, mult, sigmaOnf)
    
    H = tf.zeros(E0[0].shape)
    for k in range(1, nscales + 1):
        E1 = E0[k - 1]

        cond_0 = tf.math.logical_and(tf.math.real(E1) <= 0, tf.math.imag(E1) <= 0)
        cond_1 = tf.math.logical_and(tf.math.real(E1) <= 0, tf.math.imag(E1) > 0)
        cond_2 = tf.math.logical_and(tf.math.real(E1) > 0, tf.math.imag(E1) <= 0)
        cond_3 = tf.math.logical_and(tf.math.real(E1) > 0, tf.math.imag(E1) > 0)

        H=tf.where(cond_0,0.0,H)
        H=tf.where(cond_1,1.0,H)
        H=tf.where(cond_2,2.0,H)
        H=tf.where(cond_3,3.0,H)

    return H

def tf_GaborBitStreamSTACKED(AR): #polarImage):

    #AR, AG, AB = tf.split(polarImage, num_or_size_splits=3, axis=-1)

    nscales = 1
    minWaveLength = 24
    mult = 1
    sigmaOnf = 0.5

    TR = tf_encode(tf.squeeze(AR), nscales, minWaveLength, mult, sigmaOnf)
    #TG = tf_encode(tf.squeeze(AG), nscales, minWaveLength, mult, sigmaOnf)
    #TB = tf_encode(tf.squeeze(AB), nscales, minWaveLength, mult, sigmaOnf)

    TR = tf.cast(TR, dtype=tf.uint8)

    return tf.expand_dims(TR, axis=2) #return tf.concat([tf.expand_dims(TR, axis=2) , tf.expand_dims(TG, axis=2), tf.expand_dims(TB, axis=2)], axis=-1)

def tf_daugman_feature_extractor(inp):
    return tf_GaborBitStreamSTACKED(inp)

In [ ]:
def load_image_test(image_file):
        
    input_image, real_image = load(image_file)
    _, norm_real_image = normalize2(input_image, real_image)
    
    norm_R, _, _ = tf.split(norm_real_image, num_or_size_splits=3, axis=-1)

    fi_real_image = tf_daugman_feature_extractor(norm_R)

    return norm_R, fi_real_image

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(testset_files)
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(1)

In [ ]:
def generator_loss(target, gen_output):
    # PSNR 
    psnr_loss = tf.image.psnr(target, gen_output, max_val=2.0) # images shuld have been normalized in range [-1,1]. Thus, the difference between the min and max should be 2.
    
    # SSIM 
    ssim_loss = tf.image.ssim(target, gen_output, max_val=2.0)

    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    # Mean squared error
    l2_loss = tf.reduce_mean(tf.abs(target - gen_output)**2)
    
    return psnr_loss.numpy().item(0), ssim_loss.numpy().item(0), l1_loss.numpy(), l2_loss.numpy()


def create_mask(pred_mask):
 pred_mask = tf.argmax(pred_mask, axis=-1)
 pred_mask = pred_mask[..., tf.newaxis]
 return pred_mask

def DiceLoss(targets, inputs, smooth=1e-6):
    # convert the tensor to one-hot for multi-class segmentation
    #y_true = K.squeeze(y_true, 3)
    #y_true = tf.cast(y_true, "int32")
    #y_true = tf.one_hot(y_true, 4, axis=-1)
    
    # cast to float32 datatype
    #y_true = K.cast(y_true, 'float32')
    #y_pred = K.cast(y_pred, 'float32')
    
    #flatten label and prediction tensors
    #inputs = K.flatten(y_pred)
    #targets = K.flatten(y_true)

    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice


#Tensorflow / Keras 
def IoULoss(targets, inputs, smooth=1e-6):


    intersection = K.sum(targets * inputs)
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU


def segmentation_loss(y_true, y_pred) :

    # convert the tensor to one-hot for multi-class segmentation
    mask_prediction = create_mask(y_pred)
    mask_prediction = tf.cast(mask_prediction, tf.uint8)

    y_true_m = K.squeeze(y_true, 3)
    y_true_m = tf.cast(y_true_m, "int32")
    y_true_m1 = tf.one_hot(y_true_m, 4, axis=-1)

    mask_prediction_m = K.squeeze(mask_prediction, 3)
    mask_prediction_m = tf.cast(mask_prediction_m, "int32")
    mask_prediction_m1 = tf.one_hot(mask_prediction_m, 4, axis=-1)  
    
    # cast to float32 datatype
    y_true_m = K.cast(y_true_m1, 'float32')
    y_pred_m = K.cast(y_pred, 'float32')
    mask_prediction_m =  K.cast(mask_prediction_m1, 'float32')
    
    #flatten label and prediction tensors
    y_pred_m = K.flatten(y_pred_m)
    y_true_m = K.flatten(y_true_m)
    mask_prediction_m = K.flatten(mask_prediction_m)


    #accuracy   -   precision   -   recall 

    accuracy = tf.keras.metrics.Accuracy()
    accuracy.reset_states()

    precision = tf.keras.metrics.Precision()
    precision.reset_states()

    recall = tf.keras.metrics.Recall()
    recall.reset_states()

    accuracy.update_state(y_true_m, mask_prediction_m)
    precision.update_state(y_true_m, mask_prediction_m)
    recall.update_state(y_true_m, mask_prediction_m)

    # f1 score

    precision_result = precision.result().numpy()
    recall_result = recall.result().numpy()
    f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)

    # log-loss

    cce = tf.keras.metrics.CategoricalCrossentropy()
    cce.reset_states()
    cce.update_state(y_true_m1,mask_prediction_m1)

    #kld_loss

    kld = tf.keras.metrics.KLDivergence()
    kld.reset_states()
    kld.update_state(y_true_m1, y_pred)
    
    # poisson_loss

    poisson = tf.keras.metrics.Poisson()
    poisson.reset_states()
    poisson.update_state(y_true_m, y_pred_m)

    #dice_loss_with_label

    dice_loss = DiceLoss(y_true_m, mask_prediction_m)
    
    #dice_loss_with_probability 

    iou_loss =  IoULoss(y_true_m, mask_prediction_m)

    return accuracy.result().numpy(), kld.result().numpy(), poisson.result().numpy(), dice_loss, iou_loss, cce.result().numpy(), precision_result, recall_result, f1_score

In [ ]:
for model_name in models_list :
    generator = tf.keras.models.load_model(MODELS_PATH + model_name + '.h5', compile=False)

    sum_l1_losses = [] 
    sum_l2_losses = []
    sum_psnr_losses = []
    sum_ssim_losses = []
    sum_acc = []
    sum_kld_loss = [] 
    sum_poisson_loss= []
    sum_dice_loss = [] 
    sum_iou_loss = [] 
    sum_logloss = []
    sum_precision = []
    sum_recall = []
    sum_f1score = []

    for inp, tar in test_dataset :
        gen_output = generator(inp, training=False)

        acc, kld_loss, poisson_loss, dice_loss, iou_loss, logloss, precision, recall, f1score = segmentation_loss(tar, gen_output)

        sum_acc.append(acc)
        sum_kld_loss.append(kld_loss)
        sum_poisson_loss.append(poisson_loss)
        sum_dice_loss.append(dice_loss)
        sum_iou_loss.append(iou_loss)
        sum_logloss.append(logloss)
        sum_precision.append(precision)
        sum_recall.append(recall)
        sum_f1score.append(f1score)

    print("\t  acc ")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_acc)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_acc)))
    print("\t  precision")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_precision)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_precision)))
    print("\t  recall")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_recall)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_recall)))
    print("\t  f1 score")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_f1score)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_f1score)))
    print("\t  cce")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_logloss)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_logloss)))
    print("\t  poisson loss")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_poisson_loss)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_poisson_loss)))
    print("\t  dice loss with label")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_dice_loss)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_dice_loss)))
    print("\t  iou loss with label")
    print("\t\t  mean  : " +  "{:.2f}".format(np.mean(sum_iou_loss)) +  "\tvar : " +  "{:.2f}".format(np.var(sum_iou_loss)))
    print("\n\n")

In [ ]:
# label balancing test

mean_0 =[]
mean_1 =[]
mean_2 =[]
mean_3 =[]

for inp, tar in test_dataset :
    label_0 = tar[tar==0]
    label_1 = tar[tar==1]
    label_2 = tar[tar==2]
    label_3 = tar[tar==3]

    mean_0.append(label_0.shape[0])
    mean_1.append(label_1.shape[0])
    mean_2.append(label_2.shape[0])
    mean_3.append(label_3.shape[0])

print('npixel label 0: ', np.mean(mean_0))
print('npixel label 1: ', np.mean(mean_1))
print('npixel label 2: ', np.mean(mean_2))
print('npixel label 3: ', np.mean(mean_3))